Вариант 8 (как в аудитории)

In [1]:
X_sequence = 'AATGAC'
Y_sequence = 'ATG'

# Первая часть

In [2]:
tau = 0.1
delta = 0.2
eps = 0.1
q = 0.25

In [3]:
a_MM = 1 - 2 * delta - tau   # 0.5
a_MX = delta                 # 0.2
a_MY = delta                 # 0.2

a_XM = 1 - eps - tau         # 0.8
a_XX = eps                   # 0.1
a_XE = tau                   # 0.1

a_YM = 1 - eps - tau         # 0.8
a_YY = eps                   # 0.1
a_YE = tau                   # 0.1

a_ME = tau                   # 0.1

In [4]:
probs = {
    'AA': 0.5, 'TT': 0.5, 'GG': 0.5, 'CC': 0.5,
    'CT': 0.05, 'TC': 0.05,
    'AG': 0.05, 'GA': 0.05,
    'AT': 0.3, 'TA': 0.3,
    'GC': 0.3, 'CG': 0.3,
    'GT': 0.15, 'TG': 0.15,
    'AC': 0.15, 'CA': 0.15
}

In [5]:
class Cell():
    def __init__(self):
        self.M = 0.0
        self.X = 0.0
        self.Y = 0.0
        self.bp_M = None
        self.bp_X = None
        self.bp_Y = None

    def setM(self, val, bp=None):
        self.M = val
        self.bp_M = bp

    def setX(self, val, bp=None):
        self.X = val
        self.bp_X = bp

    def setY(self, val, bp=None):
        self.Y = val
        self.bp_Y = bp


In [6]:
class Matrix():
    def __init__(self, x_seq, y_seq):
        self.X_sequence = x_seq
        self.Y_sequence = y_seq
        self.matrix = []

    def createEmptyMatrix(self):
        rows = len(self.X_sequence) + 1
        cols = len(self.Y_sequence) + 1
        self.matrix = [[Cell() for _ in range(cols)] for _ in range(rows)]

    def setStartState(self):
        self.matrix[0][0].setM(1.0)

    def computeMatrix(self):
        len_x = len(self.X_sequence) + 1
        len_y = len(self.Y_sequence) + 1

        y = 0
        for x in range(1, len_x):

            from_X = self.matrix[x-1][y].X * a_XX
            from_M = self.matrix[x-1][y].M * a_MX
            if from_X >= from_M:
                best = from_X
                bp = (x-1, y, 'X')
            else:
                best = from_M
                bp = (x-1, y, 'M')
            self.matrix[x][y].setX(q * best, bp)

        x = 0
        for y in range(1, len_y):
            from_Y = self.matrix[x][y-1].Y * a_YY
            from_M = self.matrix[x][y-1].M * a_MY
            if from_Y >= from_M:
                best = from_Y
                bp = (x, y-1, 'Y')
            else:
                best = from_M
                bp = (x, y-1, 'M')
            self.matrix[x][y].setY(q * best, bp)

        for x in range(1, len_x):
            for y in range(1, len_y):
                sx = self.X_sequence[x-1]
                sy = self.Y_sequence[y-1]
                pair = sx + sy
                emit = probs.get(pair, 0.0)


                cand_M_from_M = self.matrix[x-1][y-1].M * a_MM
                cand_M_from_X = self.matrix[x-1][y-1].X * a_XM
                cand_M_from_Y = self.matrix[x-1][y-1].Y * a_YM
                if cand_M_from_M >= cand_M_from_X and cand_M_from_M >= cand_M_from_Y:
                    bestM = cand_M_from_M
                    bpM = (x-1, y-1, 'M')
                elif cand_M_from_X >= cand_M_from_Y:
                    bestM = cand_M_from_X
                    bpM = (x-1, y-1, 'X')
                else:
                    bestM = cand_M_from_Y
                    bpM = (x-1, y-1, 'Y')
                self.matrix[x][y].setM(emit * bestM, bpM)

                from_X = self.matrix[x-1][y].X * a_XX
                from_M = self.matrix[x-1][y].M * a_MX
                if from_X >= from_M:
                    bestX = from_X
                    bpX = (x-1, y, 'X')
                else:
                    bestX = from_M
                    bpX = (x-1, y, 'M')
                self.matrix[x][y].setX(q * bestX, bpX)

                from_Y = self.matrix[x][y-1].Y * a_YY
                from_M = self.matrix[x][y-1].M * a_MY
                if from_Y >= from_M:
                    bestY = from_Y
                    bpY = (x, y-1, 'Y')
                else:
                    bestY = from_M
                    bpY = (x, y-1, 'M')
                self.matrix[x][y].setY(q * bestY, bpY)

    def traceback(self):
        len_x = len(self.X_sequence) + 1
        len_y = len(self.Y_sequence) + 1
        x, y = len_x - 1, len_y - 1

        finals = [
            ('M', self.matrix[x][y].M * a_ME, self.matrix[x][y].bp_M),
            ('X', self.matrix[x][y].X * a_XE, self.matrix[x][y].bp_X),
            ('Y', self.matrix[x][y].Y * a_YE, self.matrix[x][y].bp_Y)
        ]
        current_state, best_final_val, bp = max(finals, key=lambda t: t[1])

        print(f"Chosen final state: {current_state} with final prob {best_final_val:.2e} "
              f"(cell * a(state->E))")

        alignment_X = []
        alignment_Y = []
        states_path = []

        while (x, y) != (0, 0):
            states_path.append(current_state)

            if current_state == 'M':
                alignment_X.append(self.X_sequence[x-1])
                alignment_Y.append(self.Y_sequence[y-1])
                next_bp = self.matrix[x][y].bp_M
            elif current_state == 'X':
                alignment_X.append(self.X_sequence[x-1])
                alignment_Y.append('-')
                next_bp = self.matrix[x][y].bp_X
            else:  # Y
                alignment_X.append('-')
                alignment_Y.append(self.Y_sequence[y-1])
                next_bp = self.matrix[x][y].bp_Y

            if next_bp is None:
                break
            x, y, current_state = next_bp

        states_path.reverse()
        alignment_X.reverse()
        alignment_Y.reverse()
        return ''.join(alignment_X), ''.join(alignment_Y), ''.join(states_path)

    def debug_print(self):
        rows = len(self.matrix)
        cols = len(self.matrix[0])
        print("\nDP matrix (M, X, Y):")
        header = "    " + "  ".join(['-'] + list(self.Y_sequence))
        print(header)
        for i in range(rows):
            row_label = '-' if i == 0 else self.X_sequence[i-1]
            row_items = []
            for j in range(cols):
                c = self.matrix[i][j]
                row_items.append(f"[{c.M:.2e},{c.X:.2e},{c.Y:.2e}]")
            print(f"{row_label}  " + " ".join(row_items))


In [7]:
m = Matrix(X_sequence, Y_sequence)
m.createEmptyMatrix()
m.setStartState()
m.computeMatrix()
m.debug_print()
align_x, align_y, states = m.traceback()


DP matrix (M, X, Y):
    -  A  T  G
-  [1.00e+00,0.00e+00,0.00e+00] [0.00e+00,0.00e+00,5.00e-02] [0.00e+00,0.00e+00,1.25e-03] [0.00e+00,0.00e+00,3.13e-05]
A  [0.00e+00,5.00e-02,0.00e+00] [2.50e-01,0.00e+00,0.00e+00] [1.20e-02,0.00e+00,1.25e-02] [5.00e-05,0.00e+00,6.00e-04]
A  [0.00e+00,1.25e-03,0.00e+00] [2.00e-02,1.25e-02,0.00e+00] [3.75e-02,6.00e-04,1.00e-03] [5.00e-04,2.50e-06,1.87e-03]
T  [0.00e+00,3.13e-05,0.00e+00] [3.00e-04,1.00e-03,0.00e+00] [5.00e-03,1.87e-03,1.50e-05] [2.81e-03,2.50e-05,2.50e-04]
G  [0.00e+00,7.81e-07,0.00e+00] [1.25e-06,2.50e-05,0.00e+00] [1.20e-04,2.50e-04,6.25e-08] [1.25e-03,1.41e-04,6.00e-06]
A  [0.00e+00,1.95e-08,0.00e+00] [3.13e-07,6.25e-07,0.00e+00] [6.00e-06,6.25e-06,1.56e-08] [1.00e-05,6.25e-05,3.00e-07]
C  [0.00e+00,4.88e-10,0.00e+00] [2.34e-09,1.56e-08,0.00e+00] [2.50e-08,3.00e-07,1.17e-10] [1.50e-06,1.56e-06,1.25e-09]
Chosen final state: X with final prob 1.56e-07 (cell * a(state->E))


In [8]:
print("\nAlignment:")
print("X:", align_x)
print("Y:", align_y)
print("States:", states)


Alignment:
X: AATGAC
Y: -ATG--
States: XMMMXX


# Вторая часть

In [9]:
final_prob = 1.56e-07

target_prob = 1.56e-07 * 1.08
target_prob

1.6848e-07

Попробуем увеличить вероятность match, а вероятность mismatch уменьшить
Методом перебора были получены такие пропорции для изменения:

In [10]:
A = 1.025   # +8%
B = 0.975   # рассчитанный коэффициент

probs = {
    'AA': 0.5 * A, 'TT': 0.5 * A, 'GG': 0.5 * A, 'CC': 0.5 * A,
    'CT': 0.05 * B, 'TC': 0.05 * B,
    'AG': 0.05 * B, 'GA': 0.05 * B,
    'AT': 0.3 * B, 'TA': 0.3 * B,
    'GC': 0.3 * B, 'CG': 0.3 * B,
    'GT': 0.15 * B, 'TG': 0.15 * B,
    'AC': 0.15 * B, 'CA': 0.15 * B
}

In [11]:
m = Matrix(X_sequence, Y_sequence)
m.createEmptyMatrix()
m.setStartState()
m.computeMatrix()
m.debug_print()
align_x, align_y, states = m.traceback()


DP matrix (M, X, Y):
    -  A  T  G
-  [1.00e+00,0.00e+00,0.00e+00] [0.00e+00,0.00e+00,5.00e-02] [0.00e+00,0.00e+00,1.25e-03] [0.00e+00,0.00e+00,3.13e-05]
A  [0.00e+00,5.00e-02,0.00e+00] [2.56e-01,0.00e+00,0.00e+00] [1.17e-02,0.00e+00,1.28e-02] [4.88e-05,0.00e+00,5.85e-04]
A  [0.00e+00,1.25e-03,0.00e+00] [2.05e-02,1.28e-02,0.00e+00] [3.75e-02,5.85e-04,1.03e-03] [5.00e-04,2.44e-06,1.87e-03]
T  [0.00e+00,3.13e-05,0.00e+00] [2.93e-04,1.03e-03,0.00e+00] [5.25e-03,1.87e-03,1.46e-05] [2.74e-03,2.50e-05,2.63e-04]
G  [0.00e+00,7.81e-07,0.00e+00] [1.22e-06,2.56e-05,0.00e+00] [1.20e-04,2.63e-04,6.09e-08] [1.35e-03,1.37e-04,6.00e-06]
A  [0.00e+00,1.95e-08,0.00e+00] [3.20e-07,6.41e-07,0.00e+00] [6.00e-06,6.57e-06,1.60e-08] [1.02e-05,6.73e-05,3.00e-07]
C  [0.00e+00,4.88e-10,0.00e+00] [2.29e-09,1.60e-08,0.00e+00] [2.50e-08,3.00e-07,1.14e-10] [1.54e-06,1.68e-06,1.25e-09]
Chosen final state: X with final prob 1.68e-07 (cell * a(state->E))


In [12]:
print("\nAlignment:")
print("X:", align_x)
print("Y:", align_y)
print("States:", states)


Alignment:
X: AATGAC
Y: -ATG--
States: XMMMXX
